In [3]:
using Pkg
Pkg.add("PyCall")

Pkg.Types.PkgError: `Hwloc=0e44f5e4-bd66-52a0-8798-143a42290a1d` depends on `BinaryProvider`, but no such entry exists in the manifest.

In [1]:
using PyCall
cma = pyimport("cma")

┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1260


PyCall.PyError: PyError (PyImport_ImportModule

The Python package cma could not be found by pyimport. Usually this means
that you did not install cma in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the cma module, you can
use `pyimport_conda("cma", PKG)`, where PKG is the Anaconda
package the contains the module cma, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ModuleNotFoundError'>
ModuleNotFoundError("No module named 'cma'")


In [94]:
using Distributed
N = 1
addprocs(N*3)

3-element Array{Int64,1}:
 2
 3
 4

In [95]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

CompositeException: On worker 2:
ArgumentError: Package DrWatson not found in current path:
- Run `import Pkg; Pkg.add("DrWatson")` to install the DrWatson package.

require at .\loading.jl:892
eval at .\boot.jl:331
#101 at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Distributed\src\process_messages.jl:290
run_work_thunk at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Distributed\src\process_messages.jl:79
run_work_thunk at D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Distributed\src\process_messages.jl:88
#94 at .\task.jl:358

...and 2 more exception(s).


In [4]:
function eval_model(input_args::Array,w::Float, input_exp::Array, output_exp::Array) 
    try 
        return Catalyst.solve(input_args[1], input_args[2], 1., 
                    input_exp, output_exp, progress=false, 
                    microcomp_type=:nonlinear, 
                    Q=input_args[3], kₙ=input_args[4], 
                    calibration=true, w=w)
    catch err #in case we loose newton convergence in a microscopic computation 
        return 1e10
    end
end

function calibrate(cma,N)
    w = [1.5302e-4*(1/0.37), 1.9128e-4*(1/0.37), 2.2954e-4*(1/0.37)] # Q8, Q10, Q12
    Init = [0.006536800306, 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0], [1, 50, 20, 20]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    #Step 1
    #Read in all 3 different velocity curves
    input_exp_12 = []
    output_exp_12 = []
    input_exp_10 = []
    output_exp_10 = []
    input_exp_8 = []
    output_exp_8 = []
    
        for row in CSV.File(datadir("experiment/SBA15_interp_12.csv"); delim = ",")
            push!(input_exp_12, row.I)
            push!(output_exp_12, row.O)
        end
    
        for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = ",")
            push!(input_exp_10, row.I)
            push!(output_exp_10, row.O)
        end
    
        for row in CSV.File(datadir("experiment/SBA15_interp_8.csv"); delim = ",")
            push!(input_exp_8, row.I)
            push!(output_exp_8, row.O)
        end
    
#    microMesh = Parser.getGrid(projectdir("test/catalyst.msh"))

    #Step 2 change the model evaluation
    while isempty(es.stop()) 
        solutions = es.ask() # Returns array of array where inner array holds set of parameters length outer array = N
        #Check the syntax here
        F = Array{Future}(undef, N*3)
        errors = zeros(N*3)
        
        #Complete the for loop
        s = 1
        for idx in length(errors)
                if idx % 3 == 1
                   F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[3], input_exp_12, output_exp_12))
                    else if idx % 3 == 2
                       F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[2], input_exp_10, output_exp_10))
                    else if idx % 3 == 0
                       F[idx] = @spawnat idx+1 (eval_model(solutions[s], w[1], input_exp_8, output_exp_8))
                       s += 1
                end
        end
       
        
        # fetch the errors entry 
        for j in 1:length(errors)
            errors[j] = fetch(F[j])
        end
                
        # Reduce the N*3 length array errors to an array of length N (adding up errors)
        errors_red = sum.(Iterators.partition(errors,3))
                 
        es.tell(solutions,errors_red) 
        #This is only for saving/logging/printing current status
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 4 (seed=887702, Wed Jul 15 12:44:46 2020)
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 4 (seed=921962, Wed Jul 15 12:45:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 1.756981385054186e+02 1.0e+00 5.41e-01  5e-01  7e-01 16:39.5
    2     20 1.665959450482710e+02 1.5e+00 6.23e-01  5e-01  8e-01 32:31.5
    3     30 1.505769499928413e+02 1.7e+00 7.69e-01  6e-01  1e+00 49:01.4
    4     40 1.171577622314067e+02 2.1e+00 9.83e-01  8e-01  2e+00 66:24.9
    5     50 1.140198538930298e+02 2.5e+00 1.04e+00  8e-01  2e+00 84:31.1
    6     60 1.290544130820885e+02 2.1e+00 9.07e-01  7e-01  1e+00 102:35.2
    7     70 1.168821184065500e+02 2.1e+00 8.92e-01  6e-01  1e+00 121:26.6
    8     80 9.106513294321772e+01 2.2e+00 8.72e-01  6e-01  1e+00 139:54.8
    9     90 8.905030212783787e+01 2.4e+00 7.88e-01  5e-01  9e-01 158:33.6
   10    100 5.439316169528274e+01 2.2e+00 7.52e-01  5e-01  9e-01 177:10.8
 

In [12]:
A = [[1.,2,3],[4,5,6]]

2-element Array{Array{Float64,1},1}:
 [1.0, 2.0, 3.0]
 [4.0, 5.0, 6.0]

In [73]:
errors

15-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15

In [91]:
errors = collect(1:15)
errors_red = sum.(Iterators.partition(errors,3))

5-element Array{Int64,1}:
  6
 15
 24
 33
 42

1